Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf


#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = '2020.SR'

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo('2020.SR', '2020-11-08', '2021-11-08')
df = df.iloc[:, :4]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2020-11-08,77.500000,78.000000,77.400002,78.000000
2020-11-09,78.000000,79.500000,77.900002,79.500000
2020-11-10,80.300003,81.400002,79.900002,80.099998
2020-11-11,80.500000,80.500000,79.400002,80.000000
2020-11-12,79.599998,80.300003,79.599998,79.699997
...,...,...,...,...
2021-11-01,162.399994,166.000000,162.199997,164.800003
2021-11-02,164.800003,166.399994,163.600006,165.000000
2021-11-03,165.000000,167.399994,163.399994,165.800003


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] = df['Open'] - df['Open'].shift(1)
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2020-11-26,80.800003,80.800003,80.400002,80.599998,80.160000,-0.799995
2020-11-29,80.599998,81.099998,80.400002,80.400002,80.320000,-0.200005
2020-11-30,80.400002,82.000000,80.000000,82.000000,80.486666,-0.199997
2020-12-01,81.900002,81.900002,80.300003,81.500000,80.580000,1.500000
2020-12-02,81.000000,81.099998,80.500000,81.000000,80.646666,-0.900002
...,...,...,...,...,...,...
2021-11-01,162.399994,166.000000,162.199997,164.800003,171.760001,-1.400009
2021-11-02,164.800003,166.399994,163.600006,165.000000,171.226667,2.400009
2021-11-03,165.000000,167.399994,163.399994,165.800003,170.640000,0.199997


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(df['Close'].shift(-1) - df['Close'] >= 0, 1, -1)
y

array([-1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1,  1,  1, -1,  1, -1, -1,
        1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1,  1, -1, -1,  1, -1,  1, -1,  1,  1, -1,  1,  1, -1,  1,  1,
        1,  1, -1,  1,  1,  1, -1,  1,  1, -1, -1,  1, -1,  1, -1,  1, -1,
       -1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1,  1, -1,
        1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1,  1,  1,
        1, -1,  1,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1,  1,  1, -1,  1,
       -1, -1,  1, -1,  1, -1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,
        1, -1, -1,  1, -1, -1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,
       -1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1, -1,
       -1, -1, -1,  1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1,  1,  1,
        1,  1,  1, -1,  1, -1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,
        1,  1,  1, -1,  1,  1, -1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1,
       -1,  1,  1,  1, -1

In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train, X_test, y_train, y_test = X[:index],X[index:],y[:index],y[index:]
X_train, X_test, y_train, y_test

(                  Open        High         Low       Close  Close_15_Rolling  \
 Date                                                                           
 2020-11-26   80.800003   80.800003   80.400002   80.599998         80.160000   
 2020-11-29   80.599998   81.099998   80.400002   80.400002         80.320000   
 2020-11-30   80.400002   82.000000   80.000000   82.000000         80.486666   
 2020-12-01   81.900002   81.900002   80.300003   81.500000         80.580000   
 2020-12-02   81.000000   81.099998   80.500000   81.000000         80.646666   
 ...                ...         ...         ...         ...               ...   
 2021-07-14  120.000000  121.199997  119.000000  120.000000        119.746667   
 2021-07-15  121.000000  121.800003  119.000000  120.800003        119.906668   
 2021-07-25  120.800003  122.800003  120.400002  121.800003        120.133334   
 2021-07-26  122.400002  125.000000  121.199997  125.000000        120.506668   
 2021-07-27  125.000000  128

In [7]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train, y_train)

In [8]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[-0.33479912055956723]
1,High,[0.2437865435734355]
2,Low,[0.2576745607885198]
3,Close,[-0.21643627839913596]
4,Close_15_Rolling,[0.0552938813771877]
5,Open_1_Change,[-0.034220060855180014]


In [9]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)
print(probabilities)
print(predictions)

[[0.44553754 0.55446246]
 [0.46445269 0.53554731]
 [0.37208123 0.62791877]
 [0.38561399 0.61438601]
 [0.53988147 0.46011853]
 [0.4278218  0.5721782 ]
 [0.4771916  0.5228084 ]
 [0.33882545 0.66117455]
 [0.30337337 0.69662663]
 [0.24700657 0.75299343]
 [0.30991313 0.69008687]
 [0.29485097 0.70514903]
 [0.28927113 0.71072887]
 [0.39046089 0.60953911]
 [0.2965113  0.7034887 ]
 [0.48608871 0.51391129]
 [0.33396939 0.66603061]
 [0.31054243 0.68945757]
 [0.37853748 0.62146252]
 [0.32432954 0.67567046]
 [0.3534558  0.6465442 ]
 [0.50145889 0.49854111]
 [0.2829697  0.7170303 ]
 [0.33602951 0.66397049]
 [0.39656112 0.60343888]
 [0.43713877 0.56286123]
 [0.41042382 0.58957618]
 [0.34362126 0.65637874]
 [0.30453297 0.69546703]
 [0.43966708 0.56033292]
 [0.52898881 0.47101119]
 [0.40995776 0.59004224]
 [0.39852148 0.60147852]
 [0.56850684 0.43149316]
 [0.28388686 0.71611314]
 [0.32868852 0.67131148]
 [0.42908827 0.57091173]
 [0.45471806 0.54528194]
 [0.62435357 0.37564643]
 [0.36284295 0.63715705]


In [10]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

print(metrics.confusion_matrix(y_test,predictions))

[[ 6 23]
 [ 6 37]]


In [11]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

print(logistic.score(X_test, y_test))

0.5972222222222222


In [12]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=5)
print(cross_val)
print(cross_val.mean())

[0.625      0.60416667 0.55319149 0.70212766 0.63829787]
0.6245567375886525


/Users/saadalbaz/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
